<a href="https://colab.research.google.com/github/Ali-J-Fesharaki/Applied_ML_Course/blob/main/ML-HW04/Source/p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12==24.4.*


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.6/126.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.9/810.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%mkdir -p /content/drive/MyDrive/AML_Course/HW04/
%cd /content/drive/MyDrive/AML_Course/HW04/

Mounted at /content/drive
/content/drive/MyDrive/AML_Course/HW04


In [ ]:
!gdown --id '1wdRbfo5ilvYbt3aQqIXj_o54AhpGwznD'

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wdRbfo5ilvYbt3aQqIXj_o54AhpGwznD
To: /content/drive/MyDrive/AML_Course/HW04/crash.csv
100% 85.1k/85.1k [00:00<00:00, 51.5MB/s]


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('./crash.csv')

data.drop(['Unnamed: 0', 'Name'], axis=1, inplace=True)

data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Safety'].fillna(data['Safety'].mean(), inplace=True)

label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data['Survived'] = label_encoder.fit_transform(data['Survived'])
from sklearn.model_selection import train_test_split

# Prepare the data
X = data.drop('Survived', axis=1)
y = data['Survived']


In [ ]:
import cuml
import pandas as pd
from cuml.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,make_scorer, precision_score, recall_score, accuracy_score, f1_score

# Parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10,100],
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'poly', 'linear']
}
scoring = {
    'accuracy':make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score)
}
# Initialize the cuML SVC
svm = SVC()

# Grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring=scoring, verbose=2,refit=False)

# Fit the model
grid_search.fit(X, y)

# Output the best parameters and cross-validation scores
print("Best cross-validation scores:")
print("Precision:", grid_search.cv_results_['mean_test_precision'])
print("Recall:", grid_search.cv_results_['mean_test_recall'])
print("F1_score:", grid_search.cv_results_['mean_test_f1_score'])



Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.1s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.1s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] END ......................C=100, gamma=auto, kernel=rbf; total time=   0.1s
[CV] END ......................C=100, gamma=auto, kernel=rbf; total time=   0.1s
[CV] END .....................C=100, gamma=auto, kernel=poly; total time=   0.0s
[CV] END .....................C=100, gamma=auto, kernel=poly; total time=   0.0s
[CV] END .....................C=100, gamma=auto, kernel=poly; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

[CV] END .....................C=100, gamma=auto, kernel=poly; total time=   0.1s
[CV] END .....................C=100, gamma=auto, kernel=poly; total time=   0.0s
[CV] END ...................C=100, gamma=auto, kernel=linear; total time=   0.1s
[CV] END ...................C=100, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ...................C=100, gamma=auto, kernel=linear; total time=   0.1s
[CV] END ...................C=100, gamma=auto, kernel=linear; total time=   0.0s
[CV] END ...................C=100, gamma=auto, kernel=linear; total time=   0.0s
Best cross-validation scores:
Precision: [0.         0.         0.         0.         0.         0.
 0.77017544 0.77017544 0.77017544 0.         0.         0.
 0.75960784 0.75960784 0.75960784 0.         0.         0.
 0.72893557 0.72893557 0.72893557 0.         0.         0.        ]
Recall: [0.         0.         0.         0.         0.         0.
 0.09667519 0.09667519 0.09667519 0.         0.         0.
 0.20839727 0.2083972

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import cuml
import pandas as pd
from cuml.svm import SVC
from sklearn.metrics import accuracy_score ,recall_score,precision_score,f1_score
train_data,test_data= train_test_split(data,test_size=0.2,random_state=13)
X_train = train_data.drop('Survived', axis=1)
y_train = train_data['Survived']
X_test = test_data.drop('Survived', axis=1)
y_test = test_data['Survived']


svm=SVC()
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))



Accuracy: 0.6424581005586593
Recall: 0.11594202898550725
Precision: 0.7272727272727273
F1 Score: 0.2


In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)

# Rank based on the desired metric (e.g., F1-score)
cv_results['rank_test_accuracy'] = cv_results['mean_test_accuracy'].rank(ascending=False)

# Select the best model based on the ranking of F1-score
best_index = cv_results['rank_test_accuracy'].idxmin()
best_params = cv_results.loc[best_index, 'params']

print("Best parameters:", best_params)

# Refit the best model on the whole training set using F1-score as the refit metric
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.20, random_state=13)
X_train = train_data.drop('Survived', axis=1)
y_train = train_data['Survived']
X_test=test_data.drop('Survived', axis=1)
y_test=test_data['Survived']
svm_best = SVC(**best_params)
svm_best.fit(X_train, y_train)

# Evaluate the best model on the test set
y_pred = svm_best.predict(X_test)

# Print the evaluation metrics
print("Test set evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1_score:", f1_score(y_test, y_pred))

Best parameters: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
Test set evaluation:
Accuracy: 0.6815642458100558
Precision: 0.7142857142857143
Recall: 0.2898550724637681
F1_score: 0.41237113402061853


In [ ]:
# Get the results from GridSearchCV
results = pd.DataFrame(grid_search.cv_results_)

# Display the results
results
df=results[['param_C', 'param_gamma', 'param_kernel','mean_test_accuracy','mean_test_precision','mean_test_recall','mean_test_f1_score','rank_test_accuracy','rank_test_precision','rank_test_recall','rank_test_f1_score']]
df.round(4)

,param_C,param_gamma,param_kernel,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1_score,rank_test_accuracy,rank_test_precision,rank_test_recall,rank_test_f1_score
0,0.1,scale,rbf,0.6162,0.0000,0.0000,0.0000,10,10,10,10
1,0.1,scale,poly,0.6162,0.0000,0.0000,0.0000,10,10,10,10
2,0.1,scale,linear,0.6162,0.0000,0.0000,0.0000,10,10,10,10
3,0.1,auto,rbf,0.6162,0.0000,0.0000,0.0000,10,10,10,10
4,0.1,auto,poly,0.6162,0.0000,0.0000,0.0000,10,10,10,10
5,0.1,auto,linear,0.6162,0.0000,0.0000,0.0000,10,10,10,10
6,1,scale,rbf,0.6398,0.7702,0.0967,0.1645,7,1,7,7
7,1,scale,poly,0.6398,0.7702,0.0967,0.1645,7,1,7,7
8,1,scale,linear,0.6398,0.7702,0.0967,0.1645,7,1,7,7
9,1,auto,rbf,0.6150,0.0000,0.0000,0.0000,16,10,10,10


In [ ]:
print(df.round(4).to_string())

   param_C param_gamma param_kernel  mean_test_accuracy  mean_test_precision  mean_test_recall  mean_test_f1_score  rank_test_accuracy  rank_test_precision  rank_test_recall  rank_test_f1_score
0      0.1       scale          rbf              0.6162               0.0000            0.0000              0.0000                  10                   10                10                  10
1      0.1       scale         poly              0.6162               0.0000            0.0000              0.0000                  10                   10                10                  10
2      0.1       scale       linear              0.6162               0.0000            0.0000              0.0000                  10                   10                10                  10
3      0.1        auto          rbf              0.6162               0.0000            0.0000              0.0000                  10                   10                10                  10
4      0.1        auto        